## Deliverable 2. Create a Customer Travel Destinations Map.

In [8]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
import numpy as np

# Import API key
import sys
sys.path.append("..")
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [11]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(os.path.join("..", "Weather_Database", "WeatherPy_database.csv"))
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,P,CA,50.6996,-127.4199,41.18,93,100,6.91,light rain
1,1,S,RU,52.9167,54.7500,29.12,97,100,14.00,light snow
2,2,N,RU,56.6709,124.6491,-10.34,97,100,6.89,overcast clouds
3,3,S,ZA,-33.0117,17.9442,77.05,87,62,18.03,broken clouds
4,4,L,ZM,-17.8419,25.8543,66.94,97,100,3.76,overcast clouds


In [12]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [13]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
cities_preferred_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]
cities_preferred_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,S,ZA,-33.0117,17.9442,77.05,87,62,18.03,broken clouds
7,7,G,MY,5.4112,100.3354,82.89,88,20,0.00,few clouds
12,12,C,GF,4.9333,-52.3333,82.44,84,100,4.00,overcast clouds
18,18,A,BR,-22.9661,-42.0278,89.56,58,40,8.05,scattered clouds
20,20,B,MU,-20.3428,57.7575,80.85,83,75,8.05,broken clouds


In [14]:
# 4a. Determine if there are any empty rows.
cities_preferred_df.count()

City_ID                179
City                   179
Country                179
Lat                    179
Lng                    179
Max Temp               179
Humidity               179
Cloudiness             179
Wind Speed             179
Current Description    179
dtype: int64

In [17]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = cities_preferred_df.dropna(axis="index", how="any")
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,S,ZA,-33.0117,17.9442,77.05,87,62,18.03,broken clouds
7,7,G,MY,5.4112,100.3354,82.89,88,20,0.00,few clouds
12,12,C,GF,4.9333,-52.3333,82.44,84,100,4.00,overcast clouds
18,18,A,BR,-22.9661,-42.0278,89.56,58,40,8.05,scattered clouds
20,20,B,MU,-20.3428,57.7575,80.85,83,75,8.05,broken clouds


In [18]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,S,ZA,77.05,broken clouds,-33.0117,17.9442,
7,G,MY,82.89,few clouds,5.4112,100.3354,
12,C,GF,82.44,overcast clouds,4.9333,-52.3333,
18,A,BR,89.56,scattered clouds,-22.9661,-42.0278,
20,B,MU,80.85,broken clouds,-20.3428,57.7575,
21,R,PF,76.17,broken clouds,-23.1203,-134.9692,
28,S,BR,80.19,overcast clouds,-33.5189,-53.3681,
36,I,CD,81.46,overcast clouds,-1.9500,18.2667,
37,P,ID,76.14,moderate rain,-0.8917,119.8707,
39,A,BR,84.02,scattered clouds,-9.8756,-56.0861,


In [ ]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
        
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
        print("Hotel not found...skipping.")
        hotel_df.loc[index, "Hotel Name"] = np.nan
        

In [29]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,S,ZA,77.05,broken clouds,-33.0117,17.9442,
7,G,MY,82.89,few clouds,5.4112,100.3354,
12,C,GF,82.44,overcast clouds,4.9333,-52.3333,
18,A,BR,89.56,scattered clouds,-22.9661,-42.0278,
20,B,MU,80.85,broken clouds,-20.3428,57.7575,
21,R,PF,76.17,broken clouds,-23.1203,-134.9692,
28,S,BR,80.19,overcast clouds,-33.5189,-53.3681,
36,I,CD,81.46,overcast clouds,-1.9500,18.2667,
37,P,ID,76.14,moderate rain,-0.8917,119.8707,
39,A,BR,84.02,scattered clouds,-9.8756,-56.0861,


In [30]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [33]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [36]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))